## Data clean / exploration

In [127]:
import csv
import pandas as pd
import numpy as np

import seaborn as sns

In [128]:
# import data
calendar_all = pd.read_csv('./data/calendar.csv')
listings_all = pd.read_csv('./data/listings.csv')
sentiment_scores = pd.read_csv('./data/scores.csv')
# reviews = pd.read_csv('./data/reviews.csv')


In [129]:
# do a bit of cleaning

# remove cols that only contain (1) unique elem
listings = listings_all
for col in listings_all.columns:
    if len(listings_all[col].unique()) == 1:
        listings = listings.drop(col,axis=1)

In [130]:
listings.head()

list(listings.columns.values)

['id',
 'listing_url',
 'name',
 'summary',
 'space',
 'description',
 'neighborhood_overview',
 'notes',
 'transit',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'smart_location',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
 'square_feet',
 'price',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 'cleaning_fee',
 'guests_included',
 'extra_people',
 '

In [131]:
# Trimmed Frame
trimmed_listings = listings[['id','host_id', 'host_response_rate', 'host_total_listings_count',
                             'host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
                             'guests_included', 'bathrooms', 'bedrooms', 'beds', 'price',
                               'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
                               'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                               'review_scores_location', 'review_scores_value']]

In [132]:
# populate review NA's with average
pd.options.mode.chained_assignment = None 
trimmed_listings['review_scores_rating'] = trimmed_listings.review_scores_rating.fillna(trimmed_listings.review_scores_rating.median())
trimmed_listings['review_scores_accuracy'] = trimmed_listings.review_scores_accuracy.fillna(trimmed_listings.review_scores_accuracy.median())
trimmed_listings['review_scores_cleanliness'] = trimmed_listings.review_scores_cleanliness.fillna(trimmed_listings.review_scores_cleanliness.median())
trimmed_listings['review_scores_checkin'] = trimmed_listings.review_scores_checkin.fillna(trimmed_listings.review_scores_checkin.median())
trimmed_listings['review_scores_communication'] = trimmed_listings.review_scores_communication.fillna(trimmed_listings.review_scores_communication.median())
trimmed_listings['review_scores_location'] = trimmed_listings.review_scores_location.fillna(trimmed_listings.review_scores_location.median())
trimmed_listings['review_scores_value'] = trimmed_listings.review_scores_value.fillna(trimmed_listings.review_scores_value.median())


In [133]:
# Add sentiment scores column
# Used Microsoft Azure Sentiment Text Analysis API through Postman requests to identify the sentiment score 
#  of listing descriptions from 1 - 100
trimmed_listings = trimmed_listings.join(sentiment_scores['score'], how="inner")

# rename to something better
trimmed_listings.rename(index=str, columns={"neighbourhood_cleansed": "neighbourhood"}, inplace=True)


In [134]:
trimmed_listings['price'] = trimmed_listings['price'].replace('[\$\,\.]', '', regex=True).astype(int) / 100

In [135]:
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].str.replace('%', '')
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].fillna('-1')
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].astype(int)
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].replace(-1, np.nan)
trimmed_listings['host_response_rate'] = trimmed_listings.host_response_rate.fillna(trimmed_listings.host_response_rate.mean())

In [141]:
# remove infrequent property types
trimmed_listings = trimmed_listings[trimmed_listings.property_type.isin(['Apartment', 'House' , 'Condominium',
       'Townhouse', 'Loft', 'Bed & Breakfast'])]


array(['Apartment', 'House', 'Condominium', 'Townhouse', 'Loft',
       'Bed & Breakfast'], dtype=object)

In [143]:
trimmed_listings = pd.get_dummies(trimmed_listings)

In [144]:
trimmed_listings.to_csv("./data/cleaned-listings-data")

,id,host_id,host_response_rate,host_total_listings_count,accommodates,guests_included,bathrooms,bedrooms,beds,price,...,neighbourhood_Yesler Terrace,property_type_Apartment,property_type_Bed & Breakfast,property_type_Condominium,property_type_House,property_type_Loft,property_type_Townhouse,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,241032,956883,96.000000,3.0,4,2,1.0,1.0,1.0,85.0,...,0,1,0,0,0,0,0,1,0,0
1,953595,5177328,98.000000,6.0,4,1,1.0,1.0,1.0,150.0,...,0,1,0,0,0,0,0,1,0,0
2,3308979,16708587,67.000000,2.0,11,10,4.5,5.0,7.0,975.0,...,0,0,0,0,1,0,0,1,0,0
3,7421966,9851441,94.886798,1.0,3,1,1.0,0.0,2.0,100.0,...,0,1,0,0,0,0,0,1,0,0
4,278830,1452570,100.000000,2.0,6,6,2.0,3.0,3.0,450.0,...,0,0,0,0,1,0,0,1,0,0
5,5956968,326758,94.886798,1.0,2,1,1.0,1.0,1.0,120.0,...,0,0,0,0,1,0,0,0,1,0
6,1909058,2497928,100.000000,1.0,2,1,1.0,1.0,1.0,80.0,...,0,0,0,0,1,0,0,0,1,0
8,4948745,2166277,94.886798,1.0,2,1,1.0,1.0,1.0,90.0,...,0,1,0,0,0,0,0,0,1,0
9,2493658,5177328,98.000000,6.0,4,1,1.0,1.0,1.0,150.0,...,0,1,0,0,0,0,0,1,0,0
10,175576,239585,100.000000,1.0,2,2,1.0,1.0,1.0,95.0,...,0,0,0,0,1,0,0,1,0,0
